In [1]:
import numpy, pandas,csv,datetime

In [2]:
#Create CSV ("people.csv")

header = ["id","name","gender","dob","contact","address"]
data = [[1,"Ivan","M","01-01-1818",83474578,"Blk 10"],
[2,"John Doe","M","02-02-1818",93474578,"Blk 11"],
[3,"Jane Doe","F","30-12-1899",81523477,"Blk 12"],
[4,"James Doe","M","01-01-1818",9345336,"Blk 13"]]


with open('people.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

# close the file
f.close()

#Create CSV ("my_work.csv")

header = ["id","name","date","start_time","end_time"]
data = [[1,"Handcraft Part 1","10-10-2022","09:00","14:00"],
[2,"Handcraft Part 2","10-10-2022","14:00","16:00"],
[3,"Handcraft Part 3","11-10-2022","12:00","16:00"],
[4,"Handcraft Part 4","11-10-2022","13:00","16:00"]]


with open('my_work.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

# close the file
f.close()

#Create CSV ("my_busy.csv")
header = ["id","busy_start_date","busy_start_time","busy_till_date","busy_till_time"]
data = [[2,"10-10-2022","09:30","10-10-2022","14:00"],
[2,"10-10-2022","14:00","10-10-2022","15:00"],
[3,"10-10-2022","17:00","10-10-2022","18:00"],
[4,"10-10-2022","14:00","10-10-2022","15:00"]]


with open('my_busy.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

# close the file
f.close()

In [3]:
#convert CSV files to Dataframe
my_busy = pandas.read_csv("my_busy.csv")
my_work = pandas.read_csv("my_work.csv")
people = pandas.read_csv("people.csv")

In [4]:
#convert String to Datetime format
my_busy["busy_start_datetime"] = pandas.to_datetime(my_busy["busy_start_date"].astype(str)+","+my_busy["busy_start_time"], format='%d-%m-%Y,%H:%M', errors='ignore')
my_busy["busy_end_datetime"] = pandas.to_datetime(my_busy["busy_till_date"].astype(str)+","+my_busy["busy_till_time"], format='%d-%m-%Y,%H:%M', errors='ignore')
my_busy.drop(my_busy.columns[1:-2],axis=1,inplace=True)



my_work["work_start_datetime"] = pandas.to_datetime(my_work["date"].astype(str)+","+my_work["start_time"], format='%d-%m-%Y,%H:%M', errors='ignore')
my_work["work_end_datetime"] = pandas.to_datetime(my_work["date"].astype(str)+","+my_work["end_time"], format='%d-%m-%Y,%H:%M', errors='ignore')
my_work.drop(my_work.columns[2:-2],axis=1,inplace=True)



people["dob_datetime"] = pandas.to_datetime(people["dob"].astype(str)+",00:00", format='%d-%m-%Y,%H:%M', errors='ignore')
people.drop("dob",axis=1,inplace=True)

In [5]:
my_work

,id,name,work_start_datetime,work_end_datetime
0,1,Handcraft Part 1,2022-10-10 09:00:00,2022-10-10 14:00:00
1,2,Handcraft Part 2,2022-10-10 14:00:00,2022-10-10 16:00:00
2,3,Handcraft Part 3,2022-10-11 12:00:00,2022-10-11 16:00:00
3,4,Handcraft Part 4,2022-10-11 13:00:00,2022-10-11 16:00:00


In [6]:
for item,value in my_work.iterrows():
  print('\033[4m'+value["name"] + " @ "+ value["work_start_datetime"].strftime('%d %b %Y') +" (From: " +value["work_start_datetime"].strftime('%H:%M') + " TO " + value["work_end_datetime"].strftime('%H:%M') + ')\033[0m')
  result = my_busy[~my_busy.busy_start_datetime.between(value["work_start_datetime"], value["work_end_datetime"],inclusive="neither") & \
                    ~my_busy.busy_end_datetime.between(value["work_start_datetime"], value["work_end_datetime"],inclusive="neither") | \
                   (value["work_end_datetime"] in my_busy.busy_start_datetime) | (value["work_start_datetime"] in my_busy.busy_end_datetime)] \
                   .join(people.drop(people.columns[[2,4,5]],axis=1).set_index(["id"]), on=['id']).drop(columns="id")
  print(result[result.columns[[-2,-1,0,1]]].to_string(index=False))
  print("\n")

Handcraft Part 1 @ 10 Oct 2022 (From: 09:00 TO 14:00)
     name  contact busy_start_datetime   busy_end_datetime
 John Doe 93474578 2022-10-10 14:00:00 2022-10-10 15:00:00
 Jane Doe 81523477 2022-10-10 17:00:00 2022-10-10 18:00:00
James Doe  9345336 2022-10-10 14:00:00 2022-10-10 15:00:00


Handcraft Part 2 @ 10 Oct 2022 (From: 14:00 TO 16:00)
    name  contact busy_start_datetime   busy_end_datetime
John Doe 93474578 2022-10-10 09:30:00 2022-10-10 14:00:00
Jane Doe 81523477 2022-10-10 17:00:00 2022-10-10 18:00:00


Handcraft Part 3 @ 11 Oct 2022 (From: 12:00 TO 16:00)
     name  contact busy_start_datetime   busy_end_datetime
 John Doe 93474578 2022-10-10 09:30:00 2022-10-10 14:00:00
 John Doe 93474578 2022-10-10 14:00:00 2022-10-10 15:00:00
 Jane Doe 81523477 2022-10-10 17:00:00 2022-10-10 18:00:00
James Doe  9345336 2022-10-10 14:00:00 2022-10-10 15:00:00


Handcraft Part 4 @ 11 Oct 2022 (From: 13:00 TO 16:00)
     name  contact busy_start_datetime   busy_end_datetime
 John Doe 9347